In [1]:
import sys
from pathlib import Path

In [2]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

2022-11-14 16:49:12.575910: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-14 16:49:12.730452: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-14 16:49:13.260614: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64
2022-11-14 16:49:13.260687: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: can

In [3]:
sys.path.append(str(Path("../../ssl").resolve()))

In [4]:
from src.models.pi_model.pi_model import PiModel
from src.models.pi_model.pi_model_config import PiModelConfig
from src.trainers.basic.categorical_ce import CategoricalCETrainer
from src.trainers.basic.categorical_ce_config import CategoricalCETrainerConfig
from src.data_loaders.basic.categorical_ce_config import CategoricalCEDataLoaderConfig
from src.data_loaders.basic.categorical_ce import CategoricalCEDataLoader

# Description

In this notebook, a model will be trained on the CIFAR-10 dataset using only 20% of the labelled dataset.

## Set up Experiment

In [5]:
class TrainerConfig(CategoricalCETrainerConfig):
    num_epochs = 200

train_config = TrainerConfig()

In [6]:
class ModelConfig(PiModelConfig):
    input_shape = (32, 32, 3)
    output_shape = 10

model_config = ModelConfig()

In [7]:
class DataLoaderConfig(CategoricalCEDataLoaderConfig):
    batch_size = 64
    num_classes = 10
    shuffle_buffer_size = 5000 # dataset size    
    
    # switch augmenters on for fair comparison with pi-model
    blur_params = {
        'chance': 0.10,
        'kernel_ratio': 0.10,
        'blur_strength': (0.1, 2.0)
    }

    crop_params = {
        'chance': 0.50,
        'crop_size': (0.08, 1.0),
        'aspect_range': (0.75, 1.33),
        'num_tries': 100
    }

    jitter_params = {
        'chance': 0.50,
        'distort_strength': 0.20,
        'drop_chance': 0.05
    }

data_loader_config = DataLoaderConfig()

## Get Datasets

In [8]:
(x_train_full, y_train_full), (x_test_full, y_test_full) = tf.keras.datasets.cifar10.load_data()

In [9]:
# get train dataset by subsampling 20% of the full training dataset (stratified by labels)
# add the rest as unlabelled samples
x_train_unlabelled, x_train_labelled, y_train_unlabelled_, y_train_labelled = train_test_split(
    x_train_full, y_train_full,
    stratify = y_train_full,
    test_size = 0.20, random_state = 42
)

# cast types
y_train_unlabelled = (-1 * np.ones_like(y_train_unlabelled_)).astype(np.int64)
y_train_labelled = y_train_labelled.astype(np.int64)
y_test_full = y_test_full.astype(np.int64)

In [10]:
train_data = tf.data.Dataset.from_tensor_slices((x_train_labelled, y_train_labelled))
val_data = tf.data.Dataset.from_tensor_slices((x_test_full, y_test_full))

2022-11-14 16:49:15.236786: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 16:49:15.244791: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 16:49:15.245396: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 16:49:15.246641: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [11]:
# create train dataset
train_data = CategoricalCEDataLoader(train_data, data_loader_config)(training = True)

In [12]:
# create test dataset
val_data = CategoricalCEDataLoader(val_data, data_loader_config)(training = False)

In [13]:
print(f"Train dataset size: {train_data.cardinality()}")
print(f"Validation dataset size: {val_data.cardinality()}")

Train dataset size: 157
Validation dataset size: 157


# Train Model

In [14]:
model = PiModel(model_config)()

In [15]:
trainer = CategoricalCETrainer(
    model, train_data, train_config,
    val_dataset = val_data)

In [16]:
trainer.train()

2022-11-14 16:49:17.455672: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204


Training loss at epoch 0 is : 2.18. Validation loss is : 1.97. Validation acc. is : 23.63.
Training loss at epoch 1 is : 1.99. Validation loss is : 1.91. Validation acc. is : 27.88.
Training loss at epoch 2 is : 1.91. Validation loss is : 1.83. Validation acc. is : 28.97.
Training loss at epoch 3 is : 1.86. Validation loss is : 1.73. Validation acc. is : 37.00.
Training loss at epoch 4 is : 1.82. Validation loss is : 1.77. Validation acc. is : 35.22.
Training loss at epoch 5 is : 1.77. Validation loss is : 1.61. Validation acc. is : 41.05.
Training loss at epoch 6 is : 1.76. Validation loss is : 1.62. Validation acc. is : 39.57.
Training loss at epoch 7 is : 1.73. Validation loss is : 1.56. Validation acc. is : 41.74.
Training loss at epoch 8 is : 1.70. Validation loss is : 1.55. Validation acc. is : 44.07.
Training loss at epoch 9 is : 1.66. Validation loss is : 1.54. Validation acc. is : 44.15.
Training loss at epoch 10 is : 1.65. Validation loss is : 1.47. Validation acc. is : 47.28